In [ ]:
# !pip freeze > requirements.txt

In [ ]:
#DEPENDENCIES

!pip install fastapi uvicorn
!pip install colabcode
!pip install mtcnn
!pip install python-multipart

In [ ]:
import keras
keras.__version__

'2.8.0'

In [ ]:
# Authorizing ngrok terminal

!ngrok authtoken #auth_token

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# Importing Necessary modules

from fastapi import FastAPI, File, UploadFile
import uvicorn
from colabcode import ColabCode
import os
from keras.models import load_model
import pickle
from mtcnn.mtcnn import MTCNN
import numpy as np
from typing import List
from PIL import Image
from io import BytesIO
from sklearn.preprocessing import LabelEncoder

In [ ]:

# extract a single face from a given photograph
# EXTRACT FACES
def extract_face(img, required_size=(160, 160)):
    print('extracting face')

    image = img.convert('RGB')
    pixels = np.asarray(image)
    print('image converted to array')

    # detect face using MTCNN
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    
    # extract the bounding box from the first face
    try:
        x1, y1, width, height = results[0]['box']
    
        # bug fix
        x1, y1 = abs(x1), abs(y1)

        x2, y2 = x1 + width, y1 + height

        face = pixels[y1:y2, x1:x2]

        image = Image.fromarray(face)

        image = image.resize(required_size)
        face_array = np.asarray(image)

        return face_array

    except Exception as e:
        print(e)
        print('exception occcured')

    # required_size=(160, 160)
    image = image.resize(required_size) 
    face_array = np.asarray(image)

    print('extracted face')
    return face_array



# Create Face Embeddings using pre-trained model contributed by Hiroki Taniai
def get_embedding(model, face_pixels):
    print('embedding image called')

    face_pixels = face_pixels.astype('float32')
    
    # standardize pixel values across channels (global) 
    mean, std = face_pixels.mean(), face_pixels.std() 
    face_pixels = (face_pixels - mean) / std
    samples = np.expand_dims(face_pixels, axis=0)
    
    # get face embedding using pre-trained FaceNet model
    yhat = model.predict(samples)
    print('Predicted embedding')
    return yhat[0]

## Driver code to pre_process an image
## READ AS BYTES ARRAY-->EXTRACT FACE-->PREDICT FACE EMBEDDING-->RESHAPE ARRAY
def preprocess_img(file):

  print('PREPROCESSING IMAGE.......')

  image = Image.open(BytesIO(file))

  # EXTRACTS FACE FROM IMAGE
  inp = extract_face(image)  

  #MAKES EMBEDDING OF THE IMAGE USING PRE-TRAINED MODEL
  inp_emb = get_embedding(em_model, inp) 
  print('PREPROCESSING done!!!!!!!!')
  return inp_emb.reshape(1,-1)


In [ ]:
app = FastAPI(
    title='Face Predictor using FaceNet',
    description="<ul><li>Select the <b>'/predict'</b> method and click <b>'try it out'</b></li><li>Upload an image of a person</li><li>click on <b>'execute'</b>,scroll down to see the result in response body</li>"
    )

model = None
em_model = None
encoder = LabelEncoder()

## Load models on startup or first run of API
@app.on_event("startup")
def load_models():
  global model
  global em_model
  global encoder
  # Loading model for face prediction
  model = pickle.load(open('/content/drive/MyDrive/FaceNet_API/Models/model.pkl', 'rb'))

  # Loading model for predicting face embeddings by Hiroki Taniai
  em_model = load_model('/content/drive/MyDrive/FaceNet_API/Models/facenet_keras.h5')

  # Loading labels
  encoder.classes_ = np.load('/content/drive/MyDrive/FaceNet_API/Models/classes.npy')

  print('MODELS LOADED !!!\n')

## Instructions TO USE API when link is opened
@app.get('/',include_in_schema=False)
def index():
    return {
        'Message':'Hello!',
        '1)':'redirect to [link_generated]/docs',
        '2)':'Select upload method',
        '3)':'Click on Try it Out',
        '4)':'Upload an image and click execute',
        '5)':'After loading is complete you can view the results in Responses section'
        }


## Prediction is done: 
## File upload -> Execute -> Get name, distance, Registration status
@app.post("/predict")
async def prediction(file: UploadFile = File(...)):

    # Read file
    contents = await file.read()

    # extract face from img -> Get face embedding -> Reshape array
    face = preprocess_img(contents)
    # Predict face (employee name)
    face_index = model.predict(face)
    # Predict distance of input from available classes
    prob = model.predict_proba(face)
    print('PREDICTION DONE !!!!!!')

    # Calculating index of most probable distance and mapping to respective name using encoder
    prob = np.max(prob)*100
    predicted = encoder.inverse_transform(face_index)

    # Setting THRESHOLD as 30
    #msg = 'Face Registered'
    #if prob>90: msg = 'Face Not registered'
    
    return {
        #'message':{msg},
        'face':{predicted[0]},
        'Distance':{prob}
        }

In [ ]:
# Running the app on cloud server
# Use Public URL: NgrokTunnel
# Add "/docs" to the generated link i.e. [link]/docs
# Example: https://317d-35-196-110-71.ngrok.io/docs

server = ColabCode(port=8000, code=False)
server.run_app(app=app)


Public URL: NgrokTunnel: "https://fda9-35-196-110-71.ngrok.io" -> "http://localhost:8000"


INFO:     Started server process [1406]
INFO:     Waiting for application startup.


INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


MODELS LOADED !!!

INFO:     2405:201:c008:226a:79f1:e7b9:d813:1669:0 - "GET / HTTP/1.1" 200 OK
INFO:     2405:201:c008:226a:79f1:e7b9:d813:1669:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2405:201:c008:226a:79f1:e7b9:d813:1669:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2405:201:c008:226a:79f1:e7b9:d813:1669:0 - "GET /openapi.json HTTP/1.1" 200 OK
PREPROCESSING IMAGE.......
extracting face
image converted to array
embedding image called
Predicted embedding
PREPROCESSING done!!!!!!!!
PREDICTION DONE !!!!!!
INFO:     2405:201:c008:226a:79f1:e7b9:d813:1669:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1406]
